# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](https://nbviewer.jupyter.org/github/NREL-SIIP/SIIPExamples.jl/blob/master/notebook/3_PowerSimulations_examples/02_sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies
### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
using PowerSystemCaseBuilder

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [2]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data

In [3]:
sys = build_system(PSITestSystems, "test_RTS_GMLC_sys")

┌ Info: Building new system test_RTS_GMLC_sys from raw data
└   sys_descriptor.raw_data = "/Users/cbarrows/.julia/packages/PowerSystemCaseBuilder/LRCSW/data/RTS_GMLC"
[ Info: Parsing csv files in Reserves ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R3.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Down.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Up.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R3.csv ...
[ Info: Success

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [4]:
for line in get_components(Line, sys)
    if (get_base_voltage(get_from(get_arc(line))) >= 230.0) &&
       (get_base_voltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

[ Info: Changing C31-2 to MonitoredLine
┌ Warning: Rate 500.0 MW for C31-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
[ Info: Changing B26 to MonitoredLine
┌ Warning: Rate 500.0 MW for B26 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
[ Info: Changing A32-2 to MonitoredLine
┌ Warning: Rate 500.0 MW for A32-2 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
[ Info: Changing CA-1 to MonitoredLine
┌ Warning: Rate 500.0 MW for CA-1 is larger than the max expected in the range of (min = 134.0, max = 145.0).
└ @ PowerSystems ~/.julia/packages/PowerSystems/61h6O/src/utils/IO/branchdata_checks.jl:148
[ Info: Changing B34 to MonitoredL

Let's start with a standard unit commitment template using the `PTDFPowerModel` network
formulation which only constructs the admittance matrix rows corresponding to "bounded" lines:

In [5]:
template = template_unit_commitment(transmission = PTDFPowerModel)


Operations Problem Specification
Transmission: CopperPlatePowerModel
Devices Models: 

	Type: ThermalStandard
 	Formulation: ThermalBasicUnitCommitment

	Type: HydroDispatch
 	Formulation: HydroDispatchRunOfRiver

	Type: PowerLoad
 	Formulation: StaticPowerLoad

	Type: RenewableFix
 	Formulation: FixedOutput

	Type: RenewableDispatch
 	Formulation: RenewableFullDispatch

	Type: HydroEnergyReservoir
 	Formulation: HydroDispatchRunOfRiver

	Type: InterruptibleLoad
 	Formulation: InterruptiblePowerLoad

Branches Models: 

	Type: Line
 	Formulation: StaticBranch

	Type: TapTransformer
 	Formulation: StaticBranch

	Type: Transformer2W
 	Formulation: StaticBranch

	Type: HVDCLine
 	Formulation: HVDCDispatch

Services Models:

	Type: VariableReserve{ReserveDown}
 	Formulation: RangeReserve

	Type: VariableReserve{ReserveUp}
 	Formulation: RangeReserve



Notice that there is no entry for `MonitoredLine`, so we can add one:

In [6]:
set_device_model!(template, MonitoredLine, StaticBranch)

We can also relax the formulation applied to the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [7]:
set_device_model!(template, Line, StaticBranchUnbounded)

[ Info: Overwriting Line existing model


## Build an `OperationsProblem`

In [8]:
uc_prob = OperationsProblem(template, sys, horizon = 24, optimizer = solver)
build!(uc_prob, output_dir = mktempdir())

BuildStatus.BUILT = 0

Solve the relaxed problem

In [9]:
solve!(uc_prob)

RunStatus.SUCCESSFUL = 0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*